## 8.1 ネットワークをより深く

In [1]:
import os
import sys
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [2]:
os.chdir('/Users/yuta.shimizu/Downloads/Machine Learning/deep-learning-from-scratch-master/ch08')
sys.path.append(os.pardir)

In [3]:
from deep_convnet import DeepConvNet
from collections import OrderedDict
from common.layers import *
from dataset.mnist import load_mnist
from common.trainer import Trainer

### 実装

In [4]:
class DeepConvNet:
    
    def __init__(
        self, input_dim=(1, 28, 28),
        conv_param_1 = {'filter_num': 16, 'filter_size': 3, 'pad':1, 'stride':1},
        conv_param_2 = {'filter_num': 16, 'filter_size': 3, 'pad':1, 'stride':1},
        conv_param_3 = {'filter_num': 32, 'filter_size': 3, 'pad':1, 'stride':1},
        conv_param_4 = {'filter_num': 32, 'filter_size': 3, 'pad':2, 'stride':1},
        conv_param_5 = {'filter_num': 64, 'filter_size': 3, 'pad':1, 'stride':1},
        conv_param_6 = {'filter_num': 64, 'filter_size': 3, 'pad':1, 'stride':1},
        hidden_size=50, output_size=10
    ):
    
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        weight_init_scales = np.sqrt(2.0 / pre_node_nums)
        
        self.params = {}
        pre_channel_num = input_dim[0]
        
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = weight_init_scales[idx] * np.random.randn(
                conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size']
            )
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        
        self.params['W7'] = weight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = weight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)
        
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        
        self.layers.append(Convolution(self.params['W4'], self.params['b4'], conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        
        self.layers.append(Convolution(self.params['W5'], self.params['b5'], conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        
        self.layers.append(Convolution(self.params['W6'], self.params['b6'], conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)
    
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
            
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)
            
        return acc / x.shape[0]
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.last_layer.backward(dout)
        
        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)
            
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db
            
        return grads
        
    def save_params(self, file_name='params.pkl'):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)
            
    def load_params(self, file_name='params.pkl'):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val
        
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

`np.random.randn` は、行列の各次元を引数にとっている。

In [5]:
np.random.randn(3, 3)

array([[ 1.19682237, -0.49356639, -0.66541329],
       [ 1.29621773,  0.20940981,  2.07949145],
       [-0.97495517, -0.47047955, -0.45910464]])

`layer_idx` は、`self.layers` に格納された Conv のインデックスを指定している。

### 学習

In [6]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()
trainer = Trainer(
    network, x_train, t_train, x_test, t_test, epochs=20, mini_batch_size=100,
    optimizer='Adam', optimizer_param={'lr': 0.001}, evaluate_sample_num_per_epoch=1000
)
trainer.train()

network.save_params('deep_convnet_params.pkl')
print('Save Network Parameters!')

train loss:2.357024600117992
=== epoch:1, train acc:0.108, test acc:0.114 ===
train loss:2.3565978978238373
train loss:2.2837450887762305
train loss:2.274793375367765
train loss:2.285169250587435
train loss:2.26923548142803
train loss:2.2864202221507015
train loss:2.25572972728607
train loss:2.225601584099485
train loss:2.228667828662848
train loss:2.234840127486413
train loss:2.285592357971904
train loss:2.2162907144535797
train loss:2.2195429493766827
train loss:2.199444758830077
train loss:2.174498514001645
train loss:2.261666374433676
train loss:2.119572230641124
train loss:2.1681419817301792
train loss:2.1472401316780787
train loss:2.070446587104757
train loss:2.064546749946911
train loss:2.090399933717764
train loss:2.031705803463345
train loss:2.067486990508267
train loss:2.006243749594706
train loss:2.1062286216290147
train loss:1.9447357735585111
train loss:2.069473282466495
train loss:2.089439539596811
train loss:1.951545402121392
train loss:2.019738626503189
train loss:1.958

train loss:1.2167370069635672
train loss:1.3001688537670837
train loss:1.2806353479227968
train loss:1.291937672524081
train loss:1.3713032241402763
train loss:1.3339134696942614
train loss:1.2781719501888567
train loss:1.3951612585273625
train loss:1.3015350267907335
train loss:1.3206830417229214
train loss:1.1819855936143706
train loss:1.292188548967698
train loss:1.2317345856671509
train loss:1.3870873607670686
train loss:1.1987474470623622
train loss:1.3319948923228395
train loss:1.2778671521705904
train loss:1.2506044955862095
train loss:1.4905279566923486
train loss:1.0718198309083697
train loss:1.59393064871886
train loss:1.2688630840748756
train loss:1.0945890903929978
train loss:1.3374266704501165
train loss:1.3428336241917602
train loss:1.2195816845917384
train loss:1.1963792948323877
train loss:1.3043199816817752
train loss:1.3141312596759431
train loss:1.2660269675559563
train loss:1.357193071958157
train loss:1.3838145625844638
train loss:1.2049824958173918
train loss:1.29

train loss:1.097913672072802
train loss:1.29724620119826
train loss:1.047697260096883
train loss:1.1725309466329188
train loss:1.0502111732523212
train loss:1.0491263071814456
train loss:1.219371315491473
train loss:1.0126794461811994
train loss:0.977672284472296
train loss:1.13732087679456
train loss:1.2154246252998082
train loss:1.091647967134896
train loss:1.0532119298741682
train loss:1.1649745270073966
train loss:1.0427876460693353
train loss:1.0904079558311683
train loss:1.0642143491161413
train loss:1.0618753885187748
train loss:1.1759196523320765
train loss:1.1268154780587003
train loss:1.0721077805410835
train loss:1.2340812273375692
train loss:1.0444424977716198
train loss:1.1188604821817099
train loss:1.1388566850866326
train loss:1.2626781850193445
train loss:0.951449639559886
train loss:1.0840850487731846
train loss:1.2184462918742986
train loss:1.2761488806678587
train loss:1.0824171242241838
train loss:0.9646156868027381
train loss:1.180076483185359
train loss:1.04347260

train loss:0.993984985601655
train loss:0.9616064639684165
train loss:0.9006584890969401
train loss:1.032941360565387
train loss:0.9648000805268774
train loss:1.1255995730689126
train loss:1.2158043731877675
train loss:1.1083194535459382
train loss:1.1992720266369532
train loss:1.0958520255552655
train loss:1.037510399100614
train loss:0.9767217157288435
train loss:1.02351852953205
train loss:0.8749904427161717
train loss:1.09232174836478
train loss:0.855076709155745
train loss:1.1004901480719878
train loss:0.9542198963437721
train loss:1.0289347886307783
train loss:1.0923488464129836
train loss:1.0157864495603763
train loss:0.9792482145614602
train loss:0.8356688905572303
train loss:1.0301493770991441
train loss:1.0258416196390607
train loss:0.9712932454312666
train loss:1.2075430294204366
train loss:1.1888902033286726
train loss:1.1433472816260846
train loss:0.963963572201906
train loss:1.3882318678252146
train loss:0.9659037933888177
train loss:0.8429685823427178
train loss:1.183377

train loss:1.20973272323603
train loss:0.849370761663213
train loss:1.1064110433963088
train loss:1.0724500013907001
train loss:1.037409024380724
train loss:0.9447849933692549
train loss:0.9025564897680574
train loss:1.095684658655619
train loss:0.9914736777454145
train loss:1.0044858457612422
train loss:0.8533107475148648
train loss:0.979328692488295
train loss:1.1091324041423525
train loss:1.0269860557673702
train loss:0.9965796515573337
train loss:1.0608019413422807
train loss:1.0835771724161039
train loss:0.9072681588963085
train loss:1.0072378834327516
train loss:0.8855595195983075
train loss:0.7783521520388943
train loss:0.967235716092404
train loss:0.8927420649168019
train loss:0.9858989565407384
train loss:1.099305393185223
train loss:0.9488053472952075
train loss:1.1393410037294514
train loss:1.1234042981913466
train loss:0.9680247375679475
train loss:0.8308967486480909
train loss:0.9514669814771193
train loss:1.0435095176101057
train loss:1.0197353372378684
train loss:0.93865

...

train loss:1.030350787366428
train loss:0.9092950274133571
train loss:0.8971304827248378
train loss:0.8050879839461512
train loss:0.8187987382424619
train loss:0.806680104271765
train loss:0.9127257862659657
train loss:0.8516483241473739
train loss:0.8092966050811039
train loss:0.7147082629568914
train loss:0.7992943664686933
train loss:0.9597442105713566
train loss:0.7525962493131161
train loss:0.8298624039017359
train loss:0.7819752072127667
train loss:0.8599857718131426
train loss:0.8124419778810088
train loss:0.9007320559745033
train loss:0.7182270773961545
train loss:0.8978437010548016
train loss:0.7546423920933152
train loss:0.78989871698175
train loss:0.9774907584187514
train loss:0.803384216841221
train loss:0.9438192848842837
train loss:0.9405804141266365
train loss:0.8976609785467343
train loss:0.7731527286425611
train loss:0.8850393633886309
train loss:0.8227438192666358
train loss:0.9880355093078776
train loss:0.8108930931474935
train loss:0.8226923771264297
train loss:0.69

train loss:0.8490432286592831
train loss:0.8390133016197067
train loss:0.9195739833263239
train loss:0.8285495776400296
train loss:0.8730553387470111
train loss:0.8287474545918105
train loss:0.9240926811309111
train loss:0.8229313639123343
train loss:0.796858077015772
train loss:0.8445440765177797
train loss:0.8675656608365171
train loss:0.7621912662954898
train loss:0.8966886416275633
train loss:0.8608342552758146
train loss:0.894574618359378
train loss:0.7684170717642929
train loss:0.9862683448921935
train loss:0.8871145807808531
train loss:0.9052620681106113
train loss:0.8098736930543118
train loss:0.9685086535322398
train loss:0.8670910759959566
train loss:0.7843825985617673
train loss:0.9496517649528653
train loss:0.9478773467215382
train loss:0.924808799062169
train loss:0.7338301209512954
train loss:0.9545796760494346
train loss:0.7515620869968526
train loss:0.7894633747379988
train loss:0.7350160975646799
train loss:0.9981026842705082
train loss:0.8385033637560444
train loss:0.

train loss:0.7343530507456151
train loss:0.7438764211643707
train loss:0.8103334940094975
train loss:0.9027338087972856
train loss:0.7722716567218613
train loss:0.7825203431409504
train loss:0.9433812804547829
train loss:0.728274570294059
train loss:0.8792023357674811
train loss:0.9328390125611805
train loss:0.9766640536863097
train loss:0.8089709769999054
train loss:0.8315936475993481
train loss:0.7288656252758955
train loss:0.9626350548439616
train loss:0.8434632134173491
train loss:0.8245397226654455
train loss:0.909548625851708
train loss:0.8624327917337391
train loss:0.6676488683187424
train loss:0.9294323107292247
train loss:0.7729560282809261
train loss:0.7765368514660591
train loss:0.8369727255806086
train loss:0.8532874697348789
train loss:0.6775490020932964
train loss:0.9478444510293496
train loss:0.8549939024311567
train loss:0.7312659329595934
train loss:0.7500292511266267
train loss:0.879342570290204
train loss:0.9625958117397382
train loss:0.8693879931549712
train loss:0.

train loss:0.7995331329659214
train loss:0.7852145828618114
train loss:0.8717196630274587
train loss:0.8822427065738389
train loss:0.7904723443549916
train loss:0.9299612171863871
train loss:0.9225658462045413
train loss:0.888982387439504
train loss:0.9299497676596701
train loss:0.8863924309291555
train loss:0.9047120411221262
train loss:0.7882364553824681
train loss:0.8932884454849135
train loss:0.8597696919388318
train loss:0.8609784827170169
train loss:0.9777713372822767
train loss:0.8001550590639913
train loss:0.7606826238508959
train loss:0.7962538566542946
train loss:0.7033482669680078
train loss:0.9107885141389248
train loss:0.7485777538635334
train loss:0.8547059919269933
train loss:0.8931961333694656
train loss:0.7609851811325916
train loss:0.9853107785658266
train loss:0.9302890126127322
train loss:0.9159335227443577
train loss:0.7998432110630028
train loss:0.9783915981520902
train loss:0.7910019557020402
train loss:0.883995891149625
train loss:0.8591242904280295
train loss:1

train loss:0.7697076682092314
train loss:0.951856694312813
train loss:1.003973135009129
train loss:0.9169550675895927
train loss:1.0821824669560802
train loss:0.8508528646831667
train loss:0.7335108473415292
train loss:0.7574670117624805
train loss:0.7533838645233542
train loss:0.8727749490574503
train loss:0.7688884228141979
train loss:1.0639939268210785
train loss:0.8475815173922561
train loss:0.7403164730002594
train loss:0.7746651427680407
train loss:0.7579585739635405
train loss:0.8269046571943548
train loss:0.8344344091145175
train loss:0.9289222878984323
train loss:0.9623520789007387
train loss:1.0507878149875658
train loss:1.0444902960316507
train loss:0.9025022437142854
train loss:0.9357371872005723
train loss:0.9538204310528701
train loss:0.8136627654925406
train loss:0.8164192061666503
train loss:0.779908576726896
train loss:0.8132055202684871
train loss:1.019304537322837
train loss:0.9058285445461169
train loss:0.8490353569522598
train loss:0.9573711960977745
train loss:0.9